# Protocol of experiments 06.06.2017

__Requirements:__
1. Installed [Docker](https://store.docker.com/search?type=edition&offering=community) with the internet access
1. Python3, pip
1. Installed dependencies from requirements.txt 



In [ ]:
exec(open('../tooldog/analyse/container.py').read())

In [ ]:
import urllib.request

In [ ]:
import urllib.parse

In [ ]:
import os

In [ ]:
import tarfile

In [ ]:
def write_to_file(filename, data = '', mode='w'):
    f = open(filename, mode)
    f.write(data)
    f.close()

In [ ]:
REPO_URL = "https://github.com/gem-pasteur/Integron_Finder/"

In [ ]:
ZIP_URL = urllib.parse.urljoin(REPO_URL, "archive/master.zip") # unstable

In [ ]:
WORKDIR = "/" + REPO_URL.split('/')[4] + "-master" # dir path after unzip at root, like "/Integron_Finder-master"

In [ ]:
response = urllib.request.urlopen(ZIP_URL)

In [ ]:
data = response.read()

In [ ]:
write_to_file('tool.zip', data, 'wb')

In [ ]:
CURRENT_PATH = os.path.realpath(os.getcwd())

In [ ]:
ZIP_PATH = os.path.join(CURRENT_PATH, 'tool.zip')

In [ ]:
TAR_PATH = os.path.join(CURRENT_PATH, 'tool.tar')

In [ ]:
with tarfile.open('tool.tar', mode='w') as archive:
    archive.add(ZIP_PATH, arcname="tool.zip")

In [ ]:
PYTHON_VERSION = 2 # version of python tool

In [ ]:
TOOL_NAME = 'integron_finder' # name of the tool

In [ ]:
OUTPUT_FORMAT = 'cwl' # output: cwl / xml

In [ ]:
OUTPUT_FILENAME = TOOL_NAME + "." + OUTPUT_FORMAT # output filename

In [ ]:
PYTHON_PATH = "/usr/local/lib/python3.5/dist-packages/" if PYTHON_VERSION == 3 else \
"/usr/local/lib/python2.7/dist-packages/" # PYTHONPATH, required for argparse2tool

In [ ]:
c = Container("inkuzm/tooldog-analyser-sandbox",
              "tail -f /dev/null",  # run until we will stop the container
              environment={'PYTHONPATH': PYTHON_PATH})

In [ ]:
c.put(TAR_PATH, "/")

In [ ]:
def cd(path, cmd):
    return "bash -c 'cd " + path + " && " + cmd + "'"

In [ ]:
def pip(v, cmd):
    return "pip" + str(v) + " " + cmd

In [ ]:
GEN_CMD = TOOL_NAME + " " + "--generate_cwl_tool" if OUTPUT_FORMAT == 'cwl' else \
"--generate_galaxy_xml"

In [ ]:
cwl_tool = '' 
with c:
    exe0 = c.exec("unzip /tool.zip")
    for line in exe0:
        print(line)
    
    exe1 = c.exec(cd(WORKDIR, pip(PYTHON_VERSION, "install .")))
    for line in exe1:
        print(line)
        
    exe2 = c.exec(cd(WORKDIR, pip(PYTHON_VERSION, "install argparse2tool")))
    for line in exe2:
        print(line)
        
    exe3 = c.exec(cd(WORKDIR, GEN_CMD))
    for line in exe3:
        cwl_tool += line.decode("utf-8")[:-1]
        
    write_to_file(OUTPUT_FILENAME, cwl_tool)

In [ ]:
os.remove(TAR_PATH)

In [ ]:
os.remove(ZIP_PATH)

In [ ]:
print("Success!")